# Covermap Comparison

**Author:** Adam Yang (ayang115@umd.edu)

**Description:** Compares datasets against test sets

**Crop Maps Used:** 

In [290]:
import pandas as pd
import numpy as np
import geopandas as gdp
import os
import geemap
from sklearn.metrics import classification_report 

In [291]:
import ee

ee.Authenticate()

ee.Initialize()


Successfully saved authorization token.


## **Section 0** - Setup

**Functions**

In [292]:
# Remaps classes to crop/noncrop 
def map_values(val, value_for_crop):
    if val == value_for_crop:
        return 1
    else:
        return 0

In [293]:
# Function used in map function to extract from feature collection
def rasterExtraction(image, resolution, f_collection):
    feature = image.sampleRegions(
        collection = f_collection,
        scale = resolution
    )
    return feature

In [294]:
# Creates ee.Feature from longitude and latitude coordinates from a dataframe
def create_point(row):
    geom = ee.Geometry.Point(row["lon"], row["lat"])
    prop = dict(row)

    return ee.Feature(geom, prop)

In [295]:
def report_to_row(dataset, report, df):
    new_report = pd.DataFrame(data = {"dataset": dataset, "accuracy": report["accuracy"], "crop_f1": report["1"]["f1-score"], 
    "crop_support": report["1"]["support"], "noncrop_support": report["0"]["support"], 
    "crop_precision": report["1"]["precision"], "crop_recall": report["1"]["recall"], 
    "noncrop_precision": report["0"]["precision"], "noncrop_recall": report["0"]["recall"]}, index=[0])
    
    return pd.concat([df, new_report])

**Retrieve Test Data**

In [296]:
test_files = ["Togo.csv", "Kenya.csv"]
dir = "../data/datasets/"

In [297]:
test_data = pd.DataFrame(columns=["lat", "lon", "test_class", "ee_pts", "country"])

test_set = []
for file in test_files:
    # Set dict key name
    key = file.split('.')[0]

    # Read in data and extract test values and points 
    df = pd.read_csv((dir+file))
    df = df.loc[df["subset"] == "testing"]
    df = df[["lat", "lon", "class_probability"]]

    # Create earth engine geometry points
    df["ee_pts"] = df.apply(create_point, axis=1)

    # Recast points as 1 or 0 (threshold = 0.5)
    df["test_class"] = df["class_probability"].apply(lambda x: 1 if x>=0.5 else 0)

    df["country"] = key

    test_set.append(df)

test_data = pd.concat(test_set)

In [298]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1184 entries, 1276 to 8657
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                1184 non-null   float64
 1   lon                1184 non-null   float64
 2   class_probability  1184 non-null   float64
 3   ee_pts             1184 non-null   object 
 4   test_class         1184 non-null   int64  
 5   country            1184 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 64.8+ KB


In [299]:
# Create earth engine points for section 2
test_coll = ee.FeatureCollection(test_data["ee_pts"].tolist())

**Create dataframe for results**

In [300]:
results = pd.DataFrame(columns=["dataset", "accuracy", "crop_f1", "crop_support", "noncrop_support", 
 "crop_precision", "crop_recall", "noncrop_precision", "noncrop_recall"])

## **Section 1** - Harvest Data

In [301]:
import rasterio as rio

**Harvest Togo** ([Zenodo](https://zenodo.org/record/3836629#.Y1_WxnbMJPZ))

In [302]:
harvest_togo = rio.open("../../harvest-maps/togo_cropland_v9_08032020_binary.tif")

In [303]:
harvest_togo_sampled = test_data[["lat", "lon", "test_class"]].loc[test_data["country"]=="Togo"]
harvest_togo_sampled["harvest_class"] = list(rio.sample.sample_gen(harvest_togo, zip(harvest_togo_sampled["lon"], harvest_togo_sampled["lat"])))

In [304]:
harvest_togo_sampled = harvest_togo_sampled[harvest_togo_sampled["harvest_class"]!=255]
harvest_togo_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355 entries, 1276 to 1630
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lat            355 non-null    float64
 1   lon            355 non-null    float64
 2   test_class     355 non-null    int64  
 3   harvest_class  355 non-null    object 
dtypes: float64(2), int64(1), object(1)
memory usage: 13.9+ KB


In [305]:
harvest_togo_sampled["harvest_class"] = harvest_togo_sampled["harvest_class"].apply(lambda x: x[0]) 

In [306]:
harvest_togo_report = classification_report(harvest_togo_sampled["test_class"], harvest_togo_sampled["harvest_class"], output_dict=True)

results = report_to_row("harvest_togo", harvest_togo_report, results)

**Harvest-Kenya** ([Zenodo](https://zenodo.org/record/4271144#.Y18ucXbMJPa))

In [307]:
harvest_kenya = rio.open("../../harvest-maps/kenya_cropland_binary_2019.tif")

In [308]:
harvest_kenya_sampled = test_data[["lat", "lon", "test_class"]].loc[test_data["country"]=="Kenya"]
harvest_kenya_sampled["harvest_class"] = list(rio.sample.sample_gen(harvest_kenya, zip(harvest_kenya_sampled["lon"], harvest_kenya_sampled["lat"])))

In [309]:
harvest_kenya_sampled = harvest_kenya_sampled[harvest_kenya_sampled["harvest_class"]!=255]
harvest_kenya_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 4 to 8632
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   lat            698 non-null    float64
 1   lon            698 non-null    float64
 2   test_class     698 non-null    int64  
 3   harvest_class  698 non-null    object 
dtypes: float64(2), int64(1), object(1)
memory usage: 27.3+ KB


In [310]:
# Extract int value from list
harvest_kenya_sampled["harvest_class"] = harvest_kenya_sampled["harvest_class"].apply(lambda x: x[0]) 

In [311]:
harvest_kenya_report = classification_report(harvest_kenya_sampled["test_class"], harvest_kenya_sampled["harvest_class"], output_dict=True)

results = report_to_row("harvest_kenya", harvest_kenya_report, results)

## **Section 2** - Earth Engine Accessible

**Copernicus Land Cover** ([Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description))

In [312]:
# Load copernicus data
copernicus = ee.ImageCollection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
cop_results = copernicus.select("discrete_classification").filterDate("2019-01-01", "2020-01-01").map(lambda x: rasterExtraction(x, 100, test_coll)).flatten()

In [313]:
cop_sampled = geemap.ee_to_gdf(cop_results)
cop_sampled["cop_class"] = cop_sampled["discrete_classification"].apply(lambda x: map_values(x, 40))
cop_sampled.head()

,geometry,class_probability,discrete_classification,lat,lon,cop_class
0,None,0.00,20,9.875907,1.172471,0
1,None,0.25,124,9.180105,1.374695,0
2,None,0.00,116,9.275314,1.196385,0
3,None,0.00,114,7.346575,0.705167,0
4,None,0.25,112,6.777337,0.629817,0


In [314]:
cop_sampled = pd.merge(test_data[["lat","lon","test_class", "country"]], cop_sampled, on=["lat", "lon"])
cop_sampled.head()

,lat,lon,test_class,country,geometry,class_probability,discrete_classification,cop_class
0,9.875907,1.172471,0,Togo,None,0.00,20,0
1,9.180105,1.374695,0,Togo,None,0.25,124,0
2,9.275314,1.196385,0,Togo,None,0.00,116,0
3,7.346575,0.705167,0,Togo,None,0.00,114,0
4,6.777337,0.629817,0,Togo,None,0.25,112,0


In [315]:
for country, df in cop_sampled.groupby("country"):
    report = classification_report(df["test_class"], df["cop_class"], output_dict=True)
    results = report_to_row(str("Copernicus_"+country), report, results)


**ESA World Cover** ([Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100)) 

In [316]:
esa = ee.ImageCollection("ESA/WorldCover/v100")
esa_results = esa.filterBounds(test_coll).map(lambda x: rasterExtraction(x, 10, test_coll)).flatten()

In [317]:
esa_sampled = geemap.ee_to_gdf(esa_results)
esa_sampled["esa_class"] = esa_sampled["Map"].apply(lambda x: map_values(x, 40))
esa_sampled.head()

,geometry,Map,class_probability,lat,lon,esa_class
0,None,20,0.00,9.875907,1.172471,0
1,None,20,0.25,9.180105,1.374695,0
2,None,10,0.00,9.275314,1.196385,0
3,None,10,0.00,7.346575,0.705167,0
4,None,10,0.25,6.777337,0.629817,0


In [318]:
esa_sampled = pd.merge(test_data[["lat","lon","test_class", "country"]], esa_sampled, on=["lat", "lon"])
esa_sampled.head()

,lat,lon,test_class,country,geometry,Map,class_probability,esa_class
0,9.875907,1.172471,0,Togo,None,20,0.00,0
1,9.180105,1.374695,0,Togo,None,20,0.25,0
2,9.275314,1.196385,0,Togo,None,10,0.00,0
3,7.346575,0.705167,0,Togo,None,10,0.00,0
4,6.777337,0.629817,0,Togo,None,10,0.25,0


In [319]:
for country, df in esa_sampled.groupby("country"):
    report = classification_report(df["test_class"], df["esa_class"], output_dict=True)
    results = report_to_row(str("ESA_"+country), report, results)

**GLAD Global** ([Earth Engine](https://glad.earthengine.app/view/global-cropland-dynamics))

In [320]:
glad = ee.ImageCollection("users/potapovpeter/Global_cropland_2019")
glad_results = glad.filterBounds(test_coll).map(lambda x: rasterExtraction(x, 30, test_coll)).flatten()

In [321]:
glad_sampled = geemap.ee_to_gdf(glad_results)
glad_sampled.head()

,geometry,b1,class_probability,lat,lon
0,None,0,0.00,9.875907,1.172471
1,None,0,0.25,9.180105,1.374695
2,None,0,0.00,9.275314,1.196385
3,None,0,0.00,7.346575,0.705167
4,None,0,0.25,6.777337,0.629817


In [322]:
glad_sampled = pd.merge(test_data[["lat","lon","test_class", "country"]], glad_sampled, on=["lat", "lon"])

In [323]:
for country, df in glad_sampled.groupby("country"):
    report = classification_report(df["test_class"], df["b1"], output_dict=True)
    results = report_to_row(str("GLAD_"+country), report, results)

**Evaluation Results**

In [324]:
results

,dataset,accuracy,crop_f1,crop_support,noncrop_support,crop_precision,crop_recall,noncrop_precision,noncrop_recall
0,harvest_togo,0.771831,0.736156,152,203,0.729032,0.743421,0.805000,0.793103
0,harvest_kenya,0.952722,0.968054,520,178,0.974659,0.961538,0.891892,0.926966
0,Copernicus_Kenya,0.913148,0.937824,571,258,0.925043,0.950963,0.884298,0.829457
0,Copernicus_Togo,0.695775,0.571429,152,203,0.720000,0.473684,0.686275,0.862069
0,ESA_Kenya,0.449940,0.341040,571,258,0.975207,0.206655,0.360169,0.988372
0,ESA_Togo,0.740845,0.606838,152,203,0.865854,0.467105,0.703297,0.945813
0,GLAD_Kenya,0.849216,0.885636,571,258,0.927203,0.847636,0.716612,0.852713
0,GLAD_Togo,0.707042,0.559322,152,203,0.785714,0.434211,0.682657,0.911330
